### 1. Import Libraries

In [1]:
import os
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

### 2. Import from mlcblab

In [2]:
from mlcvlab.models.nn2 import NN2
from mlcvlab.nn.losses import l2
from mlcvlab.optim.sgd import SGD
from mlcvlab.optim.adam import Adam


### 3. Set Seed

In [3]:
np.random.seed(42)

### 4. Helper functions

In [4]:
def load_dataset():
    x, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
    y = np.array(y).astype(np.int64)
    return x,y

def prepare_data(x, y):
    y_binary = (y % 2 == 0)
    return x, y_binary

def split_train_test(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=10000, random_state=25)
    return X_train, X_test, y_train, y_test

def initialize_model(X_train, X_test, y_train, y_test, hidden_layer_neurons):
    input_layer_neurons = X_train.shape[1]
    layers_dims = [input_layer_neurons + 1, hidden_layer_neurons + 1, 1]
    W = []
    
    for l in range(1, len(layers_dims)):
        multiplier = np.sqrt(1/(layers_dims[l-1])) if l == 1 else np.sqrt(2/(layers_dims[l-1]))
        W.append(np.random.randn(layers_dims[l-1], layers_dims[l]) * multiplier)

    W0 = W[0]
    W1 = W[1]
    print(f"Size of W0 : {np.shape(W0)}, Size of W1 : {np.shape(W1)}")
    
    two_layer_nn = NN2()
    two_layer_nn.layers[0].W = W0
    two_layer_nn.layers[1].W = W1
    return two_layer_nn

def train_model(model, X_train, y_train, hidden_layer_neurons, lr=0.1, R=100):
    # final_W = SGD(model, X_train, y_train, lr, R)
    final_W = Adam(model, X_train, y_train, hidden_layer_neurons, R)
    return final_W

def test_model(model, X_test, y_test, final_W):
    model.W = final_W
    model.layers[0].W = final_W[0]
    model.layers[1].W = final_W[1]
    new_test_X = np.concatenate((np.transpose(X_test), -1*np.ones((1, len(X_test)))))
    y_hat = model.nn2(new_test_X)
    y_hat = ((y_hat.T) >= 0.5) * 1
    total = np.sum(y_hat == y_test)
    return total / len(X_test)

### 5. Run the program

In [5]:

#load data
x, y = load_dataset()

#prepare data
x, y = prepare_data(x,y)

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0
y_train = np.array([y_train]).T
y_test = np.array([y_test]).T

SGD_Neurons = 500
ADAM_Neurons = 400

#initialize model
# model = initialize_model(X_train, X_test, y_train, y_test, SGD_Neurons)
model = initialize_model(X_train, X_test, y_train, y_test, ADAM_Neurons)

#training model
# final_W = train_model(model, X_train, y_train, SGD_Neurons, lr=0.1, R=100)
final_W = train_model(model, X_train, y_train, ADAM_Neurons, R=100)
print(f"Completed training model - final W : {final_W}")

#testing model
accuracy = test_model(model, X_test, y_test, final_W)
print(f"Completed testing model - Accuracy : {accuracy}")    

c:\Users\anees\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Size of W0 : (785, 401), Size of W1 : (401, 1)
ADAM optimizer starts here:
R: 0 started
127.106189705104
127.106189705104
R: 0 completed
R: 1 started
173.96311642986612
173.96207522496965
R: 1 completed
R: 2 started
134.0325402293548
159.41664204663027
R: 2 completed
R: 3 started
170.9757865878358
170.35034700529656
R: 3 completed
R: 4 started
168.23525350616097
169.56822445327566
R: 4 completed
R: 5 started
159.37373009600256
165.32749758825034
R: 5 completed
R: 6 started
137.8711061372972
150.0174994628754
R: 6 completed
R: 7 started
111.92175960469606
122.08207336805843
R: 7 completed
R: 8 started
99.09719620137615
102.2311150231225
R: 8 completed
R: 9 started
95.96203659263114
95.62368087800189
R: 9 completed
R: 10 started
94.04966584441888
93.3959745100938
R: 10 completed
R: 11 started
91.72151199293246
91.58277076774883
R: 11 completed
R: 12 started
92.20483365816206
92.62886246202395
R: 12 completed
R: 13 started
95.0563467339692
95.67462359094748
R: 13 completed
R: 14 started
9